In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
secrets_file = open("secret.txt","r")
string = secrets_file.read()
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [3]:
play_list =["37i9dQZF1DWWEJlAGA9gs0", "37i9dQZF1DWVmps5U8gHNv","37i9dQZF1DXd6tJtr4qeot", "37i9dQZF1DWWOaP4H0w5b0", "37i9dQZF1DXbITWG1ZJKYt","37i9dQZF1DWUrPBdYfoJvz", "37i9dQZF1DX0ItcQfwbKZJ", "37i9dQZF1DX26DKvjp0s9M", "37i9dQZF1DX9GRpeH4CL0S", "37i9dQZF1DX1GT5IIzDqMe", "37i9dQZF1DXatFAWyNT5ad","37i9dQZF1DX2L0iB23Enbq",'37i9dQZEVXbNG2KDcFcKOF','37i9dQZF1DWWvhKV4FBciw','37i9dQZF1DWZBCPUIUs2iR','37i9dQZF1DWZeKCadgRdKQ','37i9dQZF1DX4UtSsGT1Sbe','37i9dQZF1DXbTxeAdrVG2l','37i9dQZF1DWWQRwui0ExPn','37i9dQZF1DX3Kdv0IChEm9','37i9dQZF1DXccH49bh52dB','37i9dQZF1DXbtYAdenGE9U','37i9dQZF1DX202yvKb0g9m','37i9dQZF1DX4mnAzYEyRVc']

In [4]:
from random import randint
from time import sleep
all_tracks = []
for i in play_list:
    results = sp.user_playlist_tracks("spotify",i)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results) 
        tracks = tracks + results['items']
        sleep(randint(1,3000)/1000) 
    all_tracks= all_tracks+tracks
    

In [5]:
from pandas import json_normalize
df_alltracks=json_normalize(all_tracks)[['track.artists','track.id','track.name','track.popularity']]
print(df_alltracks.shape)

(3262, 4)


In [6]:
artists_df = pd.DataFrame(columns=['external_urls.spotify','href', 'id', 'name', 'type', 'uri','song_id', 'song_name', 'popularity' ])
for i in df_alltracks.index:
    artists_for_song = json_normalize(df_alltracks.iloc[i]['track.artists'])
    artists_for_song['song_id']    = df_alltracks.iloc[i]['track.id']         
    artists_for_song['song_name']  = df_alltracks.iloc[i]['track.name']       
    artists_for_song['popularity'] = df_alltracks.iloc[i]['track.popularity'] 
    artists_df = pd.concat([artists_df, artists_for_song], axis=0)
no_feature_df = artists_df.reset_index(drop=True)
no_feature_df.shape

(4252, 9)

In [7]:
# I want to check the method to delete repeated songs where all the elements in the row are perfectly repeated in another row

duplicate_rows = no_feature_df[no_feature_df.duplicated()]
display(duplicate_rows.sort_values(by='song_name'))
4240-83

,external_urls.spotify,href,id,name,type,uri,song_id,song_name,popularity
2619,https://open.spotify.com/artist/40Yq4vzPs9VNUr...,https://api.spotify.com/v1/artists/40Yq4vzPs9V...,40Yq4vzPs9VNUrIBG5Jr2i,The Smashing Pumpkins,artist,spotify:artist:40Yq4vzPs9VNUrIBG5Jr2i,5QLHGv0DfpeXLNFo7SFEy1,1979 - Remastered 2012,78
2452,https://open.spotify.com/artist/32vWCbZh0xZ4o9...,https://api.spotify.com/v1/artists/32vWCbZh0xZ...,32vWCbZh0xZ4o9gkz4PsEU,Dolly Parton,artist,spotify:artist:32vWCbZh0xZ4o9gkz4PsEU,4w3tQBXhn5345eUXDGBWZG,9 to 5,80
1983,https://open.spotify.com/artist/3y2cIKLjiOlp1N...,https://api.spotify.com/v1/artists/3y2cIKLjiOl...,3y2cIKLjiOlp1Np37WiUdH,Shaboozey,artist,spotify:artist:3y2cIKLjiOlp1Np37WiUdH,2FQrifJ1N335Ljm3TjTVVf,A Bar Song (Tipsy),93
3788,https://open.spotify.com/artist/2TL8gYTNgD6nXk...,https://api.spotify.com/v1/artists/2TL8gYTNgD6...,2TL8gYTNgD6nXkyuUdDrMg,Jasmine Thompson,artist,spotify:artist:2TL8gYTNgD6nXkyuUdDrMg,3c8iiZGfEammKJuWTErE5x,Ain't Nobody (Loves Me Better) (feat. Jasmine ...,74
3787,https://open.spotify.com/artist/4bL2B6hmLlMWnU...,https://api.spotify.com/v1/artists/4bL2B6hmLlM...,4bL2B6hmLlMWnUEZnorEtG,Felix Jaehn,artist,spotify:artist:4bL2B6hmLlMWnUEZnorEtG,3c8iiZGfEammKJuWTErE5x,Ain't Nobody (Loves Me Better) (feat. Jasmine ...,74
...,...,...,...,...,...,...,...,...,...
2716,https://open.spotify.com/artist/5e4Dhzv426EvQe...,https://api.spotify.com/v1/artists/5e4Dhzv426E...,5e4Dhzv426EvQe3aDb64jL,Shania Twain,artist,spotify:artist:5e4Dhzv426EvQe3aDb64jL,1wb4P4F0sxAQ2KXrRvsx6n,You're Still The One,71
2004,https://open.spotify.com/artist/45dkTj5sMRSjrm...,https://api.spotify.com/v1/artists/45dkTj5sMRS...,45dkTj5sMRSjrmBSBeiHym,Tate McRae,artist,spotify:artist:45dkTj5sMRSjrmBSBeiHym,3rUGC1vUpkDG9CZFHMur1t,greedy,95
1976,https://open.spotify.com/artist/0PCCGZ0wGLizHt...,https://api.spotify.com/v1/artists/0PCCGZ0wGLi...,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,artist,spotify:artist:0PCCGZ0wGLizHt2KZ7hhA2,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,100
3779,https://open.spotify.com/artist/1lmU3giNF3CSbk...,https://api.spotify.com/v1/artists/1lmU3giNF3C...,1lmU3giNF3CSbkVSQmLpHQ,Surf Mesa,artist,spotify:artist:1lmU3giNF3CSbkVSQmLpHQ,62aP9fBQKYKxi7PDXwcUAS,ily (i love you baby) (feat. Emilee),76


4157

In [8]:
no_feature_df.drop_duplicates(inplace=True)

In [9]:
no_feature_df.shape

(4168, 9)

In [10]:
no_feature_df.isnull().sum()

external_urls.spotify    0
href                     0
id                       0
name                     0
type                     0
uri                      0
song_id                  0
song_name                0
popularity               0
dtype: int64

In [11]:
chunks = [(i, i+100) for i in range(0, len(no_feature_df), 100)]
chunks
audio_features_list = []
for chunk in chunks:
    id_list100 = no_feature_df['song_id'][chunk[0]:chunk[1]]
    audio_features_list = audio_features_list + sp.audio_features(id_list100)
    sleep(randint(1,3000)/1000)

In [12]:
features_df = json_normalize(audio_features_list)

In [13]:
features_df= features_df.reset_index(drop='first')

In [14]:
features_df.shape

(4168, 18)

In [15]:
features_df[features_df.isnull().any(axis=1)]


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
2207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Because I'm having to rows of the song with features dataframe that are giving me null values (including the id) I will try to remove the same index from the songs without features dataframe, before chunking the features.
features_df.dropna(how = 'all', inplace=True)
features_df.shape

(4166, 18)

In [17]:
num_features_df = features_df[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness','acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms','time_signature']]

In [20]:
from sklearn.preprocessing import StandardScaler
num_features_df_scaler = StandardScaler().fit(num_features_df)
X_prep = num_features_df_scaler.transform(num_features_df)
pd.DataFrame(X_prep,columns=num_features_df.columns).head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,-1.386768,-1.061061,0.459393,-0.926614,0.826990,-0.125240,1.102244,1.049250,0.230060,-0.439221,-1.314606,-0.584413,0.245310
1,-1.386768,-1.061061,0.459393,-0.926614,0.826990,-0.125240,1.102244,1.049250,0.230060,-0.439221,-1.314606,-0.584413,0.245310
2,-1.592832,-1.163294,-1.486686,-1.850759,0.826990,-0.392001,1.360469,1.179640,-0.460828,-0.764583,0.581267,-0.665370,0.245310
3,-1.592832,-1.163294,-1.486686,-1.850759,0.826990,-0.392001,1.360469,1.179640,-0.460828,-0.764583,0.581267,-0.665370,0.245310
4,-1.316405,-1.477908,1.015415,-2.593260,-1.209204,-0.392001,1.355252,1.215859,-0.460828,-1.126918,0.319320,-0.211314,-2.082621


In [21]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=18, random_state=11)
kmeans.fit(X_prep)

C:\Users\benja\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=18, random_state=11)

In [ ]:
# import numpy as np
# K = range(2, 20)
# inertia = []

# for k in K:
#     kmeans = KMeans(n_clusters=k,
#                     random_state=1234)
#     kmeans.fit(X_prep)
#     inertia.append(kmeans.inertia_)

# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.figure(figsize=(16,8))
# plt.plot(K, inertia, 'bx-')
# plt.xlabel('k')
# plt.ylabel('inertia')
# plt.xticks(np.arange(min(K), max(K)+1, 1.0))
# plt.title('Elbow Method showing the optimal k')

#THE OPTIMAL IS THREE BUT THE SUGGESTION IS NOT ACCURATE WITH THREE CLUSTERS!

In [22]:
features_df['cluster'] = kmeans.labels_

In [23]:
features_df.cluster.unique()

array([ 3, 17, 10,  7, 12, 16,  2,  1, 11,  5,  6,  8, 13,  0, 15,  4,  9,
       14])

In [24]:
df_wft = pd.merge(left=no_feature_df,
                        right=features_df,
                        how='inner',
                        left_on='song_id',
                        right_on='id')

In [25]:
df_wft.head()

,external_urls.spotify,href,id_x,name,type_x,uri_x,song_id,song_name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type_y,id_y,uri_y,track_href,analysis_url,duration_ms,time_signature,cluster
0,https://open.spotify.com/artist/5aIqB5nVVvmFsv...,https://api.spotify.com/v1/artists/5aIqB5nVVvm...,5aIqB5nVVvmFsvSdExz408,Johann Sebastian Bach,artist,spotify:artist:5aIqB5nVVvmFsvSdExz408,17i5jLpzndlQhbS4SrTd0B,"Cello Suite No. 1 in G Major, BWV 1007: I. Pré...",70,0.275,0.157,7.0,-18.752,1.0,0.0636,0.890,0.842,0.186,0.304,73.289,audio_features,17i5jLpzndlQhbS4SrTd0B,spotify:track:17i5jLpzndlQhbS4SrTd0B,https://api.spotify.com/v1/tracks/17i5jLpzndlQ...,https://api.spotify.com/v1/audio-analysis/17i5...,152280.0,4.0,3
1,https://open.spotify.com/artist/5aIqB5nVVvmFsv...,https://api.spotify.com/v1/artists/5aIqB5nVVvm...,5aIqB5nVVvmFsvSdExz408,Johann Sebastian Bach,artist,spotify:artist:5aIqB5nVVvmFsvSdExz408,17i5jLpzndlQhbS4SrTd0B,"Cello Suite No. 1 in G Major, BWV 1007: I. Pré...",70,0.275,0.157,7.0,-18.752,1.0,0.0636,0.890,0.842,0.186,0.304,73.289,audio_features,17i5jLpzndlQhbS4SrTd0B,spotify:track:17i5jLpzndlQhbS4SrTd0B,https://api.spotify.com/v1/tracks/17i5jLpzndlQ...,https://api.spotify.com/v1/audio-analysis/17i5...,152280.0,4.0,3
2,https://open.spotify.com/artist/5Dl3HXZjG6ZOWT...,https://api.spotify.com/v1/artists/5Dl3HXZjG6Z...,5Dl3HXZjG6ZOWT5cV375lk,Yo-Yo Ma,artist,spotify:artist:5Dl3HXZjG6ZOWT5cV375lk,17i5jLpzndlQhbS4SrTd0B,"Cello Suite No. 1 in G Major, BWV 1007: I. Pré...",70,0.275,0.157,7.0,-18.752,1.0,0.0636,0.890,0.842,0.186,0.304,73.289,audio_features,17i5jLpzndlQhbS4SrTd0B,spotify:track:17i5jLpzndlQhbS4SrTd0B,https://api.spotify.com/v1/tracks/17i5jLpzndlQ...,https://api.spotify.com/v1/audio-analysis/17i5...,152280.0,4.0,3
3,https://open.spotify.com/artist/5Dl3HXZjG6ZOWT...,https://api.spotify.com/v1/artists/5Dl3HXZjG6Z...,5Dl3HXZjG6ZOWT5cV375lk,Yo-Yo Ma,artist,spotify:artist:5Dl3HXZjG6ZOWT5cV375lk,17i5jLpzndlQhbS4SrTd0B,"Cello Suite No. 1 in G Major, BWV 1007: I. Pré...",70,0.275,0.157,7.0,-18.752,1.0,0.0636,0.890,0.842,0.186,0.304,73.289,audio_features,17i5jLpzndlQhbS4SrTd0B,spotify:track:17i5jLpzndlQhbS4SrTd0B,https://api.spotify.com/v1/tracks/17i5jLpzndlQ...,https://api.spotify.com/v1/audio-analysis/17i5...,152280.0,4.0,3
4,https://open.spotify.com/artist/5aIqB5nVVvmFsv...,https://api.spotify.com/v1/artists/5aIqB5nVVvm...,5aIqB5nVVvmFsvSdExz408,Johann Sebastian Bach,artist,spotify:artist:5aIqB5nVVvmFsvSdExz408,4SFBV7SRNG2e2kyL1F6kjU,"The Well-Tempered Clavier, Book 1, BWV 846-869...",62,0.234,0.126,0.0,-25.427,1.0,0.0446,0.989,0.896,0.102,0.216,133.998,audio_features,4SFBV7SRNG2e2kyL1F6kjU,spotify:track:4SFBV7SRNG2e2kyL1F6kjU,https://api.spotify.com/v1/tracks/4SFBV7SRNG2e...,https://api.spotify.com/v1/audio-analysis/4SFB...,139307.0,4.0,3


In [26]:
df_wft = df_wft.drop_duplicates()

In [41]:
!pip3 install fuzzywuzzy

In [42]:
import pandas as pd
from fuzzywuzzy import fuzz, process
from sklearn.preprocessing import StandardScaler

In [63]:
def fuzzy_match(user_input):
    cleaned_input = user_input.lower().strip()
    song_match = process.extractOne(cleaned_input, df_wft['song_name'], scorer=fuzz.token_sort_ratio)
    artist_match = process.extractOne(cleaned_input, df_wft['name'], scorer=fuzz.token_sort_ratio)

    if song_match[1] >= 75:
        # Filter songs based on the cluster number from the best song match
        cluster_number = df_wft.loc[df_wft['song_name'] == song_match[0], 'cluster'].iloc[0]
        matched_songs = df_wft[df_wft['cluster'] == cluster_number]
    elif artist_match[1] >= 75:
        # Filter songs based on the cluster number from the best artist match
        cluster_number = df_wft.loc[df_wft['name'] == artist_match[0], 'cluster'].iloc[0]
        matched_songs = df_wft[df_wft['cluster'] == cluster_number]
    else:
        search_target = sp.search(q=f"{user_input}", limit=5) # Search Spotify for the input song/artist
        tracks = json_normalize(search_target["tracks"]["items"])
        target_id = tracks['id'].iloc[[0]]
        target_features = json_normalize(sp.audio_features(target_id))
        target_features= target_features[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness','acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms','time_signature']]
        scaled_features = num_features_df_scaler.transform(target_features)
        target_cluster = kmeans.predict(scaled_features)
        matched_songs = df_wft[df_wft['cluster'] == target_cluster[0]]

    # Randomly select a song and its artist
    random_row = matched_songs.sample(n=1)
    matched_song_name = random_row['song_name'].iloc[0]
    matched_artist_name = random_row['name'].iloc[0]

    return f"Matched Song: {matched_song_name}\nMatched Artist: {matched_artist_name}"

# Example usage:
# user_input = input("Enter a song title or artist name: ")
# result = fuzzy_match(user_input, df_wft, kmeans_model)
# print(result)


In [64]:
user_input = input("Enter a song title or artist name: ")
result = fuzzy_match(user_input)
result

Enter a song title or artist name: Made in Russia


'Matched Song: How Soon Is Now? - 2011 Remaster\nMatched Artist: The Smiths'

An example of a given the key moments of the new additions "Made in Russia" is the input that as it has a fuzzy score less than 75 then searches in Spotify and gives the track of the first 5 results. The results are filtered to the first row then match with it feature and scaled and then clusterized. Both the scaler fit and the kmeans fit are the one we used before. Finally the code gives me a random song of the same cluster 

In [61]:
# search_target = sp.search(q=f"{user_input}", limit=5) # Search Spotify for the input song/artist
# tracks = json_normalize(search_target["tracks"]["items"])
# target_id = tracks['id'].iloc[[0]]
# target_features = json_normalize(sp.audio_features(target_id))
# target_features= target_features[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness','acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms','time_signature']]
# scaled_features = num_features_df_scaler.transform(target_features)
# target_cluster = kmeans.predict(scaled_features)
# matched_songs = df_wft[df_wft['cluster'] == target_cluster[0]]
# print(target_cluster[0])
# print(matched_songs)

# random_row = matched_songs.sample(n=1)
# print(random_row)
# matched_song_name = random_row['song_name'].iloc[0]
# matched_artist_name = random_row['name'].iloc[0]
# f"Matched Song: {matched_song_name}\nMatched Artist: {matched_artist_name}"

15
                                  external_urls.spotify  \
1898  https://open.spotify.com/artist/6QDWC6QeGhvCWC...   
1953  https://open.spotify.com/artist/7EK1bQADBoqbYX...   
1958  https://open.spotify.com/artist/2yJwXpWAQOOl5X...   
1960  https://open.spotify.com/artist/4S2yOnmsWW97dT...   
1961  https://open.spotify.com/artist/0qT79UgT5tY4yu...   
...                                                 ...   
7091  https://open.spotify.com/artist/1Z4sHcpzg0xnN8...   
7098  https://open.spotify.com/artist/674oSgXMvyLNHM...   
7100  https://open.spotify.com/artist/2gFTHPd5iboV0I...   
7110  https://open.spotify.com/artist/6sKOC3XdcoRZHh...   
7131  https://open.spotify.com/artist/7DGQ1BwG33jSlc...   

                                                   href  \
1898  https://api.spotify.com/v1/artists/6QDWC6QeGhv...   
1953  https://api.spotify.com/v1/artists/7EK1bQADBoq...   
1958  https://api.spotify.com/v1/artists/2yJwXpWAQOO...   
1960  https://api.spotify.com/v1/artists/4S2yOnmsWW9

'Matched Song: The Drapery Falls\nMatched Artist: Opeth'